In [1]:
import copy
import os
import os.path as osp
import time
import warnings
import random
import numpy as np

from functools import partial
import mmcv
from mmcv.parallel import collate
import torch
from mmcv import Config, DictAction
from mmcv.parallel import MMDataParallel
from mmcv.runner import get_dist_info, init_dist
from mmcv.runner import (HOOKS, DistSamplerSeedHook, EpochBasedRunner,
                         Fp16OptimizerHook, OptimizerHook, build_optimizer)
from mmcv.utils import get_git_hash
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
from mmcv.utils import build_from_cfg


from mmdet.core import DistEvalHook, EvalHook
from mmdet import __version__
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset, build_dataloader, replace_ImageToTensor
from mmdet.models import build_detector
from mmdet.utils import collect_env, get_root_logger

/home/emasle/.local/lib/python3.11/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
cfg = Config.fromfile('./configs/scrfd/scrfd_500m.py')

In [3]:
cfg.data.train.pipeline[5].gray_prob = 0.3 # GRAY scale augmentation with 0.3 probability

In [4]:
cfg.data.train.pipeline[5]

{'type': 'PhotoMetricDistortion',
 'brightness_delta': 32,
 'contrast_range': (0.5, 1.5),
 'saturation_range': (0.5, 1.5),
 'hue_delta': 18,
 'gray_prob': 0.3}

In [5]:
distributed = False
if cfg.get('custom_imports', None):
    from mmcv.utils import import_modules_from_strings
    import_modules_from_strings(**cfg['custom_imports'])
if cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True

elif cfg.get('work_dir', None) is None:
    cfg.work_dir = osp.join('./work_dirs',
                            osp.splitext(osp.basename('./configs/scrfd/scrfd_500m.py'))[0])
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
cfg.dump(osp.join(cfg.work_dir, osp.basename('./configs/scrfd/scrfd_500m.py')))
# init the logger before other steps
timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
log_file = osp.join(cfg.work_dir, f'{timestamp}.log')
logger = get_root_logger(log_file=log_file, log_level=cfg.log_level)

meta = dict()
# log env info
env_info_dict = collect_env()
env_info = '\n'.join([(f'{k}: {v}') for k, v in env_info_dict.items()])
dash_line = '-' * 60 + '\n'
logger.info('Environment info:\n' + dash_line + env_info + '\n' +
            dash_line)
meta['env_info'] = env_info
meta['config'] = cfg.pretty_text
# log some basic info
logger.info(f'Distributed training: {distributed}')
logger.info(f'Config:\n{cfg.pretty_text}')

cfg.seed = 42
meta['seed'] = 42
meta['exp_name'] = osp.basename('./configs/scrfd/scrfd_500m.py')

2024-05-10 02:34:16,745 - mmdet - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.11.8 (main, Feb 12 2024, 14:50:05) [GCC 13.2.1 20230801]
CUDA available: True
GPU 0: NVIDIA GeForce GTX 1650
CUDA_HOME: /opt/cuda
NVCC: Cuda compilation tools, release 12.3, V12.3.107
GCC: gcc (GCC) 12.3.0
PyTorch: 2.0.1+cu117
PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.7
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-genco

In [6]:
dataset = build_dataset(cfg.data.train)

origin image size 12880


/home/emasle/code/scfrd-tz/detection/scrfd/mmdet/datasets/custom.py:156: UserWarning: CustomDataset does not support filtering empty gt images.
  warnings.warn(


In [7]:
if cfg.checkpoint_config is not None:
    # save mmdet version, config file content and class names in
    # checkpoints as meta data
    cfg.checkpoint_config.meta = dict(
        mmdet_version=__version__ + get_git_hash()[:7],
        CLASSES=dataset.CLASSES)

In [8]:
dataset.CLASSES

('FG',)

In [9]:
train_ratio = 0.8
val_ratio = 0.1

In [10]:
dataset_size = len(dataset)

random_indices = random.sample(range(dataset_size), 5000)
num_samples = len(random_indices)

In [11]:
num_train = int(train_ratio * num_samples)
num_val = int(val_ratio * num_samples)+num_train
train_split = random_indices[:num_train]
val_split = random_indices[num_train:num_val]
test_split = random_indices[num_val:]

In [12]:
print(len(train_split),len(val_split) ,len(test_split))

4000 500 500


In [13]:
train_sampler = SubsetRandomSampler(train_split)
valid_sampler = SubsetRandomSampler(val_split)
#tes_sampler = SubsetRandomSampler(test_split)

In [14]:
'''train_loader = torch.utils.data.DataLoader(dataset, batch_size=4, sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=4, sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=4, sampler=tes_sampler)'''

'train_loader = torch.utils.data.DataLoader(dataset, batch_size=4, sampler=train_sampler)\nvalidation_loader = torch.utils.data.DataLoader(dataset, batch_size=4, sampler=valid_sampler)\ntest_loader = torch.utils.data.DataLoader(dataset, batch_size=4, sampler=tes_sampler)'

In [15]:
train_loader = build_dataloader(dataset, 4,train_sampler,1,1,dist=False)
validation_loader = build_dataloader(dataset, 4,valid_sampler,1,1,dist=False)
#test_loader = build_dataloader(dataset, 4,tes_sampler,1,1,dist=False)

<h1>Trainng</h1>

In [16]:
def train_detector(model,
                   cfg,
                   dataloader,
                   val_dataloader=None,
                   distributed=False,
                   validate=False,
                   timestamp=None,
                   meta=None):
    logger = get_root_logger(cfg.log_level)

    if 'imgs_per_gpu' in cfg.data:
        cfg.data.samples_per_gpu = cfg.data.imgs_per_gpu


    model = MMDataParallel(model.cuda(), device_ids=[0])

    # build runner
    optimizer = build_optimizer(model, cfg.optimizer)
    runner = EpochBasedRunner(
        model,
        optimizer=optimizer,
        work_dir=cfg.work_dir,
        logger=logger,
        meta=meta)
    runner.timestamp = timestamp

    # fp16 setting
    fp16_cfg = cfg.get('fp16', None)
    if fp16_cfg is not None:
        optimizer_config = Fp16OptimizerHook(
            **cfg.optimizer_config, **fp16_cfg, distributed=distributed)
    elif distributed and 'type' not in cfg.optimizer_config:
        optimizer_config = OptimizerHook(**cfg.optimizer_config)
    else:
        optimizer_config = cfg.optimizer_config

    # register hooks
    runner.register_training_hooks(cfg.lr_config, optimizer_config,
                                   cfg.checkpoint_config, cfg.log_config,
                                   cfg.get('momentum_config', None))
    if distributed:
        runner.register_hook(DistSamplerSeedHook())

    # register eval hooks
    if validate:
        eval_cfg = cfg.get('evaluation', {})
        eval_hook = EvalHook
        runner.register_hook(eval_hook(val_dataloader, **eval_cfg))

    if cfg.resume_from:
        runner.resume(cfg.resume_from)
    elif cfg.load_from:
        runner.load_checkpoint(cfg.load_from)
    runner.run([dataloader], cfg.workflow, cfg.total_epochs)

In [16]:
model = build_detector(cfg.model, train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)
model.CLASSES = dataset.CLASSES

In [18]:
train_detector(
    model,
    cfg,
    train_loader,
    validation_loader,
    distributed=distributed,
    validate=True,
    timestamp=timestamp,
    meta=meta)


2024-05-08 22:53:47,179 - mmdet - INFO - Start running, host: emasle@emasle-nitroan51554, work_dir: /home/emasle/code/scfrd-tz/detection/scrfd/work_dirs/scrfd_500m
2024-05-08 22:53:47,179 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(NORMAL      ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) EvalHook                           
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) OptimizerHook                      
(NORMAL      ) CheckpointHook                    

2024-05-08 22:55:44,921 - mmdet - INFO - Epoch [1][700/1000]	lr: 4.665e-03, eta: 2:18:09, time: 0.131, data_time: 0.060, memory: 1165, loss_cls: 0.4509, loss_bbox: 1.0643, loss: 1.5152
2024-05-08 22:55:47,987 - mmdet - INFO - Epoch [1][720/1000]	lr: 4.799e-03, eta: 2:17:45, time: 0.153, data_time: 0.079, memory: 1165, loss_cls: 0.5103, loss_bbox: 1.0472, loss: 1.5575
2024-05-08 22:55:50,724 - mmdet - INFO - Epoch [1][740/1000]	lr: 4.932e-03, eta: 2:17:01, time: 0.137, data_time: 0.062, memory: 1165, loss_cls: 0.5534, loss_bbox: 1.0188, loss: 1.5722
2024-05-08 22:55:53,521 - mmdet - INFO - Epoch [1][760/1000]	lr: 5.065e-03, eta: 2:16:22, time: 0.140, data_time: 0.067, memory: 1165, loss_cls: 0.4958, loss_bbox: 1.1075, loss: 1.6033
2024-05-08 22:55:56,569 - mmdet - INFO - Epoch [1][780/1000]	lr: 5.198e-03, eta: 2:16:01, time: 0.152, data_time: 0.080, memory: 1165, loss_cls: 0.5303, loss_bbox: 0.9957, loss: 1.5260
2024-05-08 22:55:59,314 - mmdet - INFO - Epoch [1][800/1000]	lr: 5.331e-03,

2024-05-08 22:57:49,301 - mmdet - INFO - Epoch [2][580/1000]	lr: 1.000e-02, eta: 2:03:35, time: 0.130, data_time: 0.054, memory: 1517, loss_cls: 0.4923, loss_bbox: 0.9705, loss: 1.4628
2024-05-08 22:57:52,034 - mmdet - INFO - Epoch [2][600/1000]	lr: 1.000e-02, eta: 2:03:22, time: 0.137, data_time: 0.062, memory: 1517, loss_cls: 0.4669, loss_bbox: 1.0064, loss: 1.4733
2024-05-08 22:57:54,809 - mmdet - INFO - Epoch [2][620/1000]	lr: 1.000e-02, eta: 2:03:11, time: 0.139, data_time: 0.061, memory: 1517, loss_cls: 0.4725, loss_bbox: 1.0231, loss: 1.4956
2024-05-08 22:57:57,503 - mmdet - INFO - Epoch [2][640/1000]	lr: 1.000e-02, eta: 2:02:57, time: 0.135, data_time: 0.058, memory: 1517, loss_cls: 0.4422, loss_bbox: 1.0280, loss: 1.4702
2024-05-08 22:58:00,246 - mmdet - INFO - Epoch [2][660/1000]	lr: 1.000e-02, eta: 2:02:45, time: 0.137, data_time: 0.062, memory: 1517, loss_cls: 0.5252, loss_bbox: 0.9783, loss: 1.5034
2024-05-08 22:58:02,977 - mmdet - INFO - Epoch [2][680/1000]	lr: 1.000e-02,

2024-05-08 22:59:54,051 - mmdet - INFO - Epoch [3][460/1000]	lr: 1.000e-02, eta: 1:58:05, time: 0.137, data_time: 0.055, memory: 1517, loss_cls: 0.5121, loss_bbox: 0.9468, loss: 1.4589
2024-05-08 22:59:56,739 - mmdet - INFO - Epoch [3][480/1000]	lr: 1.000e-02, eta: 1:57:56, time: 0.134, data_time: 0.057, memory: 1517, loss_cls: 0.5033, loss_bbox: 0.9691, loss: 1.4724
2024-05-08 22:59:59,617 - mmdet - INFO - Epoch [3][500/1000]	lr: 1.000e-02, eta: 1:57:51, time: 0.144, data_time: 0.064, memory: 1517, loss_cls: 0.5145, loss_bbox: 0.8911, loss: 1.4056
2024-05-08 23:00:02,325 - mmdet - INFO - Epoch [3][520/1000]	lr: 1.000e-02, eta: 1:57:43, time: 0.135, data_time: 0.060, memory: 1517, loss_cls: 0.4310, loss_bbox: 1.0251, loss: 1.4561
2024-05-08 23:00:05,034 - mmdet - INFO - Epoch [3][540/1000]	lr: 1.000e-02, eta: 1:57:35, time: 0.136, data_time: 0.062, memory: 1517, loss_cls: 0.4752, loss_bbox: 0.9551, loss: 1.4303
2024-05-08 23:00:07,919 - mmdet - INFO - Epoch [3][560/1000]	lr: 1.000e-02,

2024-05-08 23:02:00,159 - mmdet - INFO - Epoch [4][340/1000]	lr: 1.000e-02, eta: 1:54:42, time: 0.138, data_time: 0.063, memory: 1643, loss_cls: 0.4651, loss_bbox: 0.8790, loss: 1.3440
2024-05-08 23:02:02,820 - mmdet - INFO - Epoch [4][360/1000]	lr: 1.000e-02, eta: 1:54:35, time: 0.133, data_time: 0.059, memory: 1643, loss_cls: 0.4776, loss_bbox: 0.8858, loss: 1.3634
2024-05-08 23:02:05,470 - mmdet - INFO - Epoch [4][380/1000]	lr: 1.000e-02, eta: 1:54:28, time: 0.133, data_time: 0.057, memory: 1643, loss_cls: 0.4170, loss_bbox: 0.9425, loss: 1.3596
2024-05-08 23:02:08,244 - mmdet - INFO - Epoch [4][400/1000]	lr: 1.000e-02, eta: 1:54:22, time: 0.138, data_time: 0.057, memory: 1643, loss_cls: 0.4190, loss_bbox: 0.9347, loss: 1.3537
2024-05-08 23:02:10,898 - mmdet - INFO - Epoch [4][420/1000]	lr: 1.000e-02, eta: 1:54:16, time: 0.133, data_time: 0.058, memory: 1643, loss_cls: 0.4847, loss_bbox: 0.8698, loss: 1.3546
2024-05-08 23:02:13,735 - mmdet - INFO - Epoch [4][440/1000]	lr: 1.000e-02,

...

2024-05-09 00:42:59,551 - mmdet - INFO - Epoch [46][700/1000]	lr: 1.000e-02, eta: 0:10:16, time: 0.148, data_time: 0.061, memory: 1643, loss_cls: 0.3133, loss_bbox: 0.5198, loss: 0.8331
2024-05-09 00:43:02,662 - mmdet - INFO - Epoch [46][720/1000]	lr: 1.000e-02, eta: 0:10:13, time: 0.155, data_time: 0.067, memory: 1643, loss_cls: 0.3023, loss_bbox: 0.5104, loss: 0.8127
2024-05-09 00:43:06,238 - mmdet - INFO - Epoch [46][740/1000]	lr: 1.000e-02, eta: 0:10:10, time: 0.179, data_time: 0.083, memory: 1643, loss_cls: 0.3047, loss_bbox: 0.5333, loss: 0.8380
2024-05-09 00:43:09,488 - mmdet - INFO - Epoch [46][760/1000]	lr: 1.000e-02, eta: 0:10:07, time: 0.162, data_time: 0.075, memory: 1643, loss_cls: 0.2831, loss_bbox: 0.5624, loss: 0.8455
2024-05-09 00:43:12,559 - mmdet - INFO - Epoch [46][780/1000]	lr: 1.000e-02, eta: 0:10:04, time: 0.154, data_time: 0.069, memory: 1643, loss_cls: 0.3318, loss_bbox: 0.5778, loss: 0.9095
2024-05-09 00:43:15,450 - mmdet - INFO - Epoch [46][800/1000]	lr: 1.00

2024-05-09 00:45:07,768 - mmdet - INFO - Epoch [47][580/1000]	lr: 1.000e-02, eta: 0:08:10, time: 0.144, data_time: 0.068, memory: 1643, loss_cls: 0.3232, loss_bbox: 0.5441, loss: 0.8672
2024-05-09 00:45:10,548 - mmdet - INFO - Epoch [47][600/1000]	lr: 1.000e-02, eta: 0:08:07, time: 0.139, data_time: 0.063, memory: 1643, loss_cls: 0.2884, loss_bbox: 0.5648, loss: 0.8532
2024-05-09 00:45:13,115 - mmdet - INFO - Epoch [47][620/1000]	lr: 1.000e-02, eta: 0:08:04, time: 0.128, data_time: 0.054, memory: 1643, loss_cls: 0.2812, loss_bbox: 0.5782, loss: 0.8595
2024-05-09 00:45:16,117 - mmdet - INFO - Epoch [47][640/1000]	lr: 1.000e-02, eta: 0:08:01, time: 0.150, data_time: 0.071, memory: 1643, loss_cls: 0.2642, loss_bbox: 0.5082, loss: 0.7724
2024-05-09 00:45:18,728 - mmdet - INFO - Epoch [47][660/1000]	lr: 1.000e-02, eta: 0:07:58, time: 0.131, data_time: 0.057, memory: 1643, loss_cls: 0.3037, loss_bbox: 0.5529, loss: 0.8566
2024-05-09 00:45:21,905 - mmdet - INFO - Epoch [47][680/1000]	lr: 1.00

2024-05-09 00:47:12,299 - mmdet - INFO - Epoch [48][460/1000]	lr: 1.000e-02, eta: 0:06:03, time: 0.135, data_time: 0.060, memory: 1643, loss_cls: 0.3163, loss_bbox: 0.5864, loss: 0.9027
2024-05-09 00:47:14,927 - mmdet - INFO - Epoch [48][480/1000]	lr: 1.000e-02, eta: 0:06:01, time: 0.131, data_time: 0.055, memory: 1643, loss_cls: 0.2933, loss_bbox: 0.6102, loss: 0.9035
2024-05-09 00:47:17,625 - mmdet - INFO - Epoch [48][500/1000]	lr: 1.000e-02, eta: 0:05:58, time: 0.135, data_time: 0.059, memory: 1643, loss_cls: 0.3006, loss_bbox: 0.5798, loss: 0.8804
2024-05-09 00:47:20,451 - mmdet - INFO - Epoch [48][520/1000]	lr: 1.000e-02, eta: 0:05:55, time: 0.141, data_time: 0.060, memory: 1643, loss_cls: 0.2782, loss_bbox: 0.5682, loss: 0.8464
2024-05-09 00:47:23,078 - mmdet - INFO - Epoch [48][540/1000]	lr: 1.000e-02, eta: 0:05:52, time: 0.131, data_time: 0.054, memory: 1643, loss_cls: 0.2892, loss_bbox: 0.5823, loss: 0.8715
2024-05-09 00:47:25,712 - mmdet - INFO - Epoch [48][560/1000]	lr: 1.00

2024-05-09 00:49:16,953 - mmdet - INFO - Epoch [49][340/1000]	lr: 1.000e-02, eta: 0:03:57, time: 0.135, data_time: 0.058, memory: 1643, loss_cls: 0.2985, loss_bbox: 0.6151, loss: 0.9136
2024-05-09 00:49:19,764 - mmdet - INFO - Epoch [49][360/1000]	lr: 1.000e-02, eta: 0:03:54, time: 0.141, data_time: 0.066, memory: 1643, loss_cls: 0.2603, loss_bbox: 0.5183, loss: 0.7786
2024-05-09 00:49:22,426 - mmdet - INFO - Epoch [49][380/1000]	lr: 1.000e-02, eta: 0:03:52, time: 0.133, data_time: 0.059, memory: 1643, loss_cls: 0.3015, loss_bbox: 0.5686, loss: 0.8701
2024-05-09 00:49:25,214 - mmdet - INFO - Epoch [49][400/1000]	lr: 1.000e-02, eta: 0:03:49, time: 0.140, data_time: 0.062, memory: 1643, loss_cls: 0.2784, loss_bbox: 0.5935, loss: 0.8719
2024-05-09 00:49:27,936 - mmdet - INFO - Epoch [49][420/1000]	lr: 1.000e-02, eta: 0:03:46, time: 0.136, data_time: 0.058, memory: 1643, loss_cls: 0.2777, loss_bbox: 0.5560, loss: 0.8337
2024-05-09 00:49:30,595 - mmdet - INFO - Epoch [49][440/1000]	lr: 1.00

2024-05-09 00:51:20,745 - mmdet - INFO - Epoch [50][220/1000]	lr: 1.000e-02, eta: 0:01:51, time: 0.135, data_time: 0.064, memory: 1643, loss_cls: 0.3551, loss_bbox: 0.4892, loss: 0.8443
2024-05-09 00:51:23,374 - mmdet - INFO - Epoch [50][240/1000]	lr: 1.000e-02, eta: 0:01:48, time: 0.132, data_time: 0.059, memory: 1643, loss_cls: 0.3040, loss_bbox: 0.5633, loss: 0.8674
2024-05-09 00:51:26,275 - mmdet - INFO - Epoch [50][260/1000]	lr: 1.000e-02, eta: 0:01:45, time: 0.145, data_time: 0.062, memory: 1643, loss_cls: 0.3288, loss_bbox: 0.5135, loss: 0.8424
2024-05-09 00:51:29,298 - mmdet - INFO - Epoch [50][280/1000]	lr: 1.000e-02, eta: 0:01:43, time: 0.151, data_time: 0.066, memory: 1643, loss_cls: 0.3129, loss_bbox: 0.5955, loss: 0.9084
2024-05-09 00:51:32,523 - mmdet - INFO - Epoch [50][300/1000]	lr: 1.000e-02, eta: 0:01:40, time: 0.161, data_time: 0.078, memory: 1643, loss_cls: 0.3415, loss_bbox: 0.5287, loss: 0.8702
2024-05-09 00:51:35,278 - mmdet - INFO - Epoch [50][320/1000]	lr: 1.00

<h1>Testing</h1>

In [16]:
from mmdet.apis import multi_gpu_test, single_gpu_test
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.core.evaluation import wider_evaluation, get_widerface_gts
from mmcv.runner import (get_dist_info, init_dist, load_checkpoint,
                         wrap_fp16_model)
import copy
import os
import os.path as osp
import time
import warnings
import random
import numpy as np

from functools import partial
import mmcv
from mmcv.parallel import collate
import torch
from mmcv import Config, DictAction
from mmcv.parallel import MMDataParallel
from mmcv.runner import get_dist_info, init_dist
from mmcv.runner import (HOOKS, DistSamplerSeedHook, EpochBasedRunner,
                         Fp16OptimizerHook, OptimizerHook, build_optimizer)
from mmcv.utils import get_git_hash
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
from mmcv.utils import build_from_cfg


from mmdet.core import DistEvalHook, EvalHook
from mmdet import __version__
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset, build_dataloader, replace_ImageToTensor
from mmdet.models import build_detector
from mmdet.utils import collect_env, get_root_logger

In [17]:
cfg = Config.fromfile('configs/scrfd/scrfd_500m.py')
if cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True
cfg.model.pretrained = None
if cfg.model.get('neck'):
    if isinstance(cfg.model.neck, list):
        for neck_cfg in cfg.model.neck:
            if neck_cfg.get('rfp_backbone'):
                if neck_cfg.rfp_backbone.get('pretrained'):
                    neck_cfg.rfp_backbone.pretrained = None
    elif cfg.model.neck.get('rfp_backbone'):
        if cfg.model.neck.rfp_backbone.get('pretrained'):
            cfg.model.neck.rfp_backbone.pretrained = None

# in case the test dataset is concatenated
if isinstance(cfg.data.test, dict):
    cfg.data.test.test_mode = True
elif isinstance(cfg.data.test, list):
    for ds_cfg in cfg.data.test:
        ds_cfg.test_mode = True

gt_path = os.path.join(os.path.dirname(cfg.data.test.ann_file), 'gt')
distributed = False

# build the dataloader
samples_per_gpu = cfg.data.test.pop('samples_per_gpu', 1)
if samples_per_gpu > 1:
    # Replace 'ImageToTensor' to 'DefaultFormatBundle'
    cfg.data.test.pipeline = replace_ImageToTensor(cfg.data.test.pipeline)

gts_easy, gts_medium, gts_hard = get_widerface_gts(gt_path)
assign_stat = [0, 0]
gts_size = []


if not os.path.exists('output'):
    os.makedirs(output_folder)

In [18]:
test_dataset = build_dataset(cfg.data.test)

origin image size 3226


In [19]:
test_loader = build_dataloader(test_dataset, 1,None,1,1,dist=False, shuffle=False)

In [20]:
model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)

In [21]:
fp16_cfg = cfg.get('fp16', None)
checkpoint = load_checkpoint(model, 'work_dirs/scrfd_500m/epoch_50.pth', map_location='cpu')
if 'CLASSES' in checkpoint['meta']:
    model.CLASSES = checkpoint['meta']['CLASSES']
else:
    model.CLASSES = dataset.CLASSES

load checkpoint from local path: work_dirs/scrfd_500m/epoch_50.pth


In [22]:
model = MMDataParallel(model, device_ids=[0])
model.eval()
results = {}
output_folder = 'output'

In [23]:
prog_bar = mmcv.ProgressBar(len(test_dataset))
for i, data in enumerate(test_loader):
    with torch.no_grad():
        result = model(return_loss=False, rescale=True, **data)
    assert len(result)==1
    batch_size = 1
    result = result[0][0]
    img_metas = data['img_metas'][0].data[0][0]
    filepath = img_metas['ori_filename']
    det_scale = img_metas['scale_factor'][0]
    #print(img_metas)
    ori_shape = img_metas['ori_shape']
    img_width = ori_shape[1]
    img_height = ori_shape[0]
    _vec = filepath.split('/')
    pa, pb = _vec[-2], _vec[1]
    if pa not in results:
        results[pa] = {}
    xywh = result.copy()
    w = xywh[:,2] - xywh[:,0]
    h = xywh[:,3] - xywh[:,1]
    xywh[:,2] = w
    xywh[:,3] = h

    event_name = pa
    img_name = pb.rstrip('.jpg')
    results[event_name][img_name] = xywh

    input_height, input_width = 640, 640
    gt_hard = gts_hard[event_name][img_name]
    #print(event_name, img_name, gt_hard.shape)
    gt_bboxes = gt_hard * det_scale
    bbox_width = gt_bboxes[:,2] - gt_bboxes[:,0]
    bbox_height = gt_bboxes[:,3] - gt_bboxes[:,1]
    bbox_area = bbox_width * bbox_height
    gt_size = np.sqrt(bbox_area+0.0001)
    gts_size += list(gt_size)
    anchor_cxs = []
    anchor_cys = []
    for idx, stride in enumerate([8,16,32,64,128]):
        height = input_height // stride
        width = input_width // stride
        anchor_centers = np.stack(np.mgrid[:height, :width][::-1], axis=-1).astype(np.float32)
        anchor_centers = (anchor_centers * stride).reshape( (-1, 2) )
        anchor_cx = anchor_centers[:,0]
        anchor_cy = anchor_centers[:,1]
        anchor_cxs += list(anchor_cx)
        anchor_cys += list(anchor_cy)
    anchor_cx = np.array(anchor_cxs, dtype=np.float32)
    anchor_cy = np.array(anchor_cys, dtype=np.float32)

    num_gts = gt_bboxes.shape[0]
    num_anchors = anchor_cx.shape[0]
    anchor_cx = np.broadcast_to(anchor_cx.reshape((1,-1)), (num_gts, num_anchors)).reshape(num_anchors, num_gts)
    anchor_cy = np.broadcast_to(anchor_cy.reshape((1,-1)), (num_gts, num_anchors)).reshape(num_anchors, num_gts)
    gt_x1 = gt_bboxes[:,0]
    gt_y1 = gt_bboxes[:,1]
    gt_x2 = gt_bboxes[:,2]
    gt_y2 = gt_bboxes[:,3]
    gt_cover = np.zeros( (gt_bboxes.shape[0], ), dtype=np.float32)
    l_ = anchor_cx - gt_x1
    t_ = anchor_cy - gt_y1
    r_ = gt_x2 - anchor_cx
    b_ = gt_y2 - anchor_cy
    dist = np.stack([l_, t_, r_, b_], axis=1).min(axis=1)
    gt_dist = dist.max(axis=0)
    gt_dist  = gt_dist / gt_size
    center_thres = 0.01
    #center_thres = -0.25
    gt_cover_inds = np.where(gt_dist>center_thres)[0]
    num_assigned = len(gt_cover_inds)
    assign_stat[0] += num_gts
    assign_stat[1] += num_assigned





    for _ in range(batch_size):
        prog_bar.update()
aps = wider_evaluation(results, gt_path, 0.5)
with open(os.path.join(output_folder, 'aps'), 'w') as f:
    f.write("%f,%f,%f\n"%(aps[0],aps[1],aps[2]))
print('APS:', aps)

print('ASSIGN:', assign_stat)
gts_size = np.array(gts_size, dtype=np.float32)
gts_size = np.sort(gts_size)
assert len(gts_size)==assign_stat[0]
print(gts_size[assign_stat[0]//2])


[>>>>>>>>>>>>>>>            ] 1890/3226, 21.0 task/s, elapsed: 90s, ETA:    64s

/tmp/ipykernel_14782/672265550.py:36: RuntimeWarning: invalid value encountered in sqrt
  gt_size = np.sqrt(bbox_area+0.0001)


[>>>>>>>>>>>>>>>>>>>>>>>>>>] 3226/3226, 22.2 task/s, elapsed: 146s, ETA:     0s
easy cost 165.3330 seconds, Average Precision: 0.7776, IoU mean: 0.019
medium cost 140.1127 seconds, Average Precision: 0.6944, IoU mean: 0.019
hard cost 162.8445 seconds, Average Precision: 0.3745, IoU mean: 0.019
APS: [0.7775974751282457, 0.6944392270830897, 0.37446179905922256]
ASSIGN: [31958, 24317]
13.433869
